In [ ]:
import sys
sys.path.append('./../..')

import brax
import jax.numpy as jnp
import numpy as np
from scipy.linalg import solve_discrete_are
from lib.utils.viz import *

import mujoco
import mediapy as media

from pdb import set_trace as st
from jax.debug import breakpoint as jst

with open('/home/mukundan/Desktop/Summer_SEM/imitation_learning/lib/model/smpl_humanoid_v5.xml', 'r') as f:
  xml = f.read()

model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

mujoco.mj_forward(model, data)
renderer.update_scene(data)
# media.show_image(renderer.render())

# display_init_positions() #show init positions here TODO

mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)
data.qacc = 0  # Assert that there is no the acceleration.
mujoco.mj_inverse(model, data)
print(data.qfrc_inverse)

height_offsets = np.linspace(-0.001, 0.001, 2001)
vertical_forces = []
for offset in height_offsets:
  mujoco.mj_resetDataKeyframe(model, data, 1)
  mujoco.mj_forward(model, data)
  data.qacc = 0
  # Offset the height by `offset`.
  data.qpos[2] += offset
  mujoco.mj_inverse(model, data)
  vertical_forces.append(data.qfrc_inverse[2])

# Find the height-offset at which the vertical force is smallest.
idx = np.argmin(np.abs(vertical_forces))
best_offset = height_offsets[idx]

# # Plot the relationship.
# plt.figure(figsize=(10, 6))
# plt.plot(height_offsets * 1000, vertical_forces, linewidth=3)
# # Red vertical line at offset corresponding to smallest vertical force.
# plt.axvline(x=best_offset*1000, color='red', linestyle='--')
# # Green horizontal line at the humanoid's weight.
# weight = model.body_subtreemass[1]*np.linalg.norm(model.opt.gravity)
# plt.axhline(y=weight, color='green', linestyle='--')
# plt.xlabel('Height offset (mm)')
# plt.ylabel('Vertical force (N)')
# plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
# plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
# plt.minorticks_on()
# plt.title(f'Smallest vertical force '
#           f'found at offset {best_offset*1000:.4f}mm.')
# plt.show()

mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)
data.qacc = 0
data.qpos[2] += best_offset
qpos0 = data.qpos.copy()  # Save the position setpoint.
mujoco.mj_inverse(model, data)
qfrc0 = data.qfrc_inverse.copy()
print('desired forces:', qfrc0)

actuator_moment = np.zeros((model.nu, model.nv))
mujoco.mju_sparse2dense(
    actuator_moment,
    data.actuator_moment.reshape(-1),
    data.moment_rownnz,
    data.moment_rowadr,
    data.moment_colind.reshape(-1),
)
ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
print('control setpoint:', ctrl0)

data.ctrl = ctrl0
mujoco.mj_forward(model, data)
print('actuator forces:', data.qfrc_actuator)

nu = model.nu  # Alias for the number of actuators.
R = np.eye(nu)

nv = model.nv  # Shortcut for the number of DoFs.

# Get the Jacobian for the root body (torso) CoM.
mujoco.mj_resetData(model, data)
data.qpos = qpos0
mujoco.mj_forward(model, data)
jac_com = np.zeros((3, nv))
mujoco.mj_jacSubtreeCom(model, data, jac_com, model.body('Pelvis').id)

# Get the Jacobian for the left foot.
jac_foot = np.zeros((3, nv))
mujoco.mj_jacBodyCom(model, data, jac_foot, None, model.body('L_Ankle').id)

jac_diff = jac_com - jac_foot
Qbalance = jac_diff.T @ jac_diff

# Get all joint names.
joint_names = [model.joint(i).name for i in range(model.njnt)]

# Get indices into relevant sets of joints.
root_dofs = range(6)
body_dofs = range(6, nv)
abdomen_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
    if 'abdomen' in name
    and not 'z' in name
]
left_leg_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
    if 'left' in name
    and ('hip' in name or 'knee' in name or 'ankle' in name)
    and not 'z' in name
]
balance_dofs = abdomen_dofs + left_leg_dofs
other_dofs = np.setdiff1d(body_dofs, balance_dofs)

# Cost coefficients.
BALANCE_COST        = 1000  # Balancing.
BALANCE_JOINT_COST  = 3     # Joints required for balancing.
OTHER_JOINT_COST    = .3    # Other joints.

# Construct the Qjoint matrix.
Qjoint = np.eye(nv)
Qjoint[root_dofs, root_dofs] *= 0  # Don't penalize free joint directly.
Qjoint[balance_dofs, balance_dofs] *= BALANCE_JOINT_COST
Qjoint[other_dofs, other_dofs] *= OTHER_JOINT_COST

# Construct the Q matrix for position DoFs.
Qpos = BALANCE_COST * Qbalance + Qjoint

# No explicit penalty for velocities.
Q = np.block([[Qpos, np.zeros((nv, nv))],
              [np.zeros((nv, 2*nv))]])

# Set the initial state and control.
mujoco.mj_resetData(model, data)
data.ctrl = ctrl0
data.qpos = qpos0

# Allocate the A and B matrices, compute them.
A = np.zeros((2*nv, 2*nv))
B = np.zeros((2*nv, nu))
epsilon = 1e-6
flg_centered = True
mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)

# Solve discrete Riccati equation.
P = solve_discrete_are(A, B, Q, R)

# Compute the feedback gain matrix K.
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

# Parameters.
DURATION = 5          # seconds
FRAMERATE = 60        # Hz

# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# Allocate position difference dq.
dq = np.zeros(model.nv)

frames = []
while data.time < DURATION:
  # Get state difference dx.
  mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
  dx = np.hstack((dq, data.qvel)).T

  # LQR control law.
  data.ctrl = ctrl0 - K @ dx

  # Step the simulation.
  mujoco.mj_step(model, data)

  # Render and save frames.
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)



[ 0.00000000e+00  0.00000000e+00  6.04229003e+02  7.06099478e-02
  1.29000832e+01  0.00000000e+00  1.08297324e+02  2.63036705e+00
  7.83203048e-01  0.00000000e+00  5.49075265e+01 -3.31800986e-01
  2.36466798e-01  0.00000000e+00  1.24670410e+01  1.63402085e-01
 -1.02747029e+00  0.00000000e+00  0.00000000e+00 -3.06994060e-02
 -1.40938182e-01  0.00000000e+00  1.08297324e+02 -2.62645925e+00
  7.57022261e-01  0.00000000e+00  5.49075265e+01  3.35708794e-01
  2.10286011e-01  0.00000000e+00  1.24670410e+01 -1.59494277e-01
 -1.05365108e+00  0.00000000e+00  0.00000000e+00  2.95365497e-02
 -1.45822179e-01  0.00000000e+00  3.36111302e+02  6.67021399e-02
  1.18358586e+00  0.00000000e+00  3.04387764e+02 -2.05375894e-02
  1.53586035e+00  0.00000000e+00  2.76304817e+02 -4.15997999e-02
  8.91065613e+00  0.00000000e+00 -7.57436068e+03  2.56554534e+00
  3.13583237e+02 -2.00571973e+00 -7.58384501e+03  0.00000000e+00
  1.13686838e-13  0.00000000e+00  0.00000000e+00  1.91041231e+01
  1.54152993e+00  0.00000

AttributeError: module 'mujoco' has no attribute 'mju_band2dense'